In [ ]:
!pip install tensorflow==2.11
!pip install tensorflow-hub==0.12


In [37]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

data_root  = ('/content/drive/My Drive/data')
IMAGE_SHAPE = (425, 425)
TRAINING_DATA_DIR = str(data_root)
datagen_kwargs = dict(rescale=1./255, validation_split=.20)
valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
valid_generator = valid_datagen.flow_from_directory(TRAINING_DATA_DIR,subset="validation",shuffle=True,target_size=(224, 224))
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
train_generator = train_datagen.flow_from_directory(TRAINING_DATA_DIR,subset="training",shuffle=True,target_size=(224, 224))

In [ ]:
for image_batch, label_batch in train_generator:
  break
image_batch.shape, label_batch.shape

In [ ]:
print (train_generator.class_indices)

In [ ]:
labels = '\n'.join(sorted(train_generator.class_indices.keys()))
with open('labels.txt', 'w') as f:
 f.write(labels)
!cat labels.txt

In [42]:
import tensorflow_hub as hub
m = keras.Sequential([
    keras.layers.Resizing(224, 224),
    hub.KerasLayer("https://www.kaggle.com/models/google/mobilenet-v3/TensorFlow2/large-075-224-classification/1"),  # Can be True, see below.
    keras.layers.Dense(train_generator.num_classes, activation='softmax')
])
m.build([None, 425, 425, 3])  # Batch input shape.


In [43]:
from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=1e-3)
m.compile(
 optimizer=optimizer,
 loss='categorical_crossentropy',
 metrics=['acc'])

In [ ]:
steps_per_epoch = np.ceil(train_generator.samples/train_generator.batch_size)
val_steps_per_epoch = np.ceil(valid_generator.samples/valid_generator.batch_size)
hist = m.fit(
 train_generator,
 epochs=100,
 verbose=1,
 steps_per_epoch=steps_per_epoch,
 validation_data=valid_generator,
 validation_steps=val_steps_per_epoch).history

In [ ]:
final_loss, final_accuracy = m.evaluate(valid_generator, steps = val_steps_per_epoch)
print("Final loss: {:.2f}".format(final_loss))
print("Final accuracy: {:.2f}%".format(final_accuracy * 100))

In [ ]:
plt.figure()
plt.ylabel("Loss (training and validation)")
plt.xlabel("Training Steps")
plt.ylim([0,50])
plt.plot(hist["loss"])
plt.plot(hist["val_loss"])
plt.figure()
plt.ylabel("Accuracy (training and validation)")
plt.xlabel("Training Steps")
plt.ylim([0,1])
plt.plot(hist["acc"])
plt.plot(hist["val_acc"])

In [ ]:
images, true_labels = next(valid_generator)

tf_model_predictions = m.predict(images)

print("Prediction results shape:", tf_model_predictions.shape)

plt.figure(figsize=(10, 9))
plt.subplots_adjust(hspace=0.5)

for n in range(min(9, len(images))):
    plt.subplot(6, 5, n + 1)
    plt.imshow(images[n])
    predicted_class = np.argmax(tf_model_predictions[n])
    true_class = np.argmax(true_labels[n])
    color = "green" if predicted_class == true_class else "red"
    plt.title(f"Pred: {predicted_class}, True: {true_class}", color=color)
    plt.axis('off')

_ = plt.suptitle("Model predictions (green: correct, red: incorrect)")
plt.show()

In [48]:
m.save('model/needle_or_not.h5')

In [50]:
from tensorflow.keras.utils import get_custom_objects

get_custom_objects().update({'KerasLayer': hub.KerasLayer})

loaded_model = tf.keras.models.load_model('model/needle_or_not.h5')

In [ ]:
from tensorflow.keras.preprocessing import image

img_path = 'test/5.png'
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0

pred = loaded_model.predict(img_array)
predicted_class = "Needle"
if np.argmax(pred, axis=1) == 1 :
  predicted_class = "Not Needle"

print(f"Object is {predicted_class}")